In [1]:
import os

os.environ["CONFIG_APP_DIR"] = "config"


In [2]:
import sys
import os
import pickle
os.chdir("..")
from omegaconf import OmegaConf

import json
from etl_pipeline.data_processor_engine.json_engine.json_engine import JsonProcessingEngine
from etl_pipeline.custom.ms.payload_loader import PayloadLoader

/env/ds/anaconda/envs/pipeline/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
from etl_pipeline.config import pipeline_config
cn = pipeline_config.cn

In [4]:
# alert = load_alert()

In [5]:
# payload = load_alert()


# Definition

In [6]:
from pipelines.ms.ms_pipeline import MSPipeline
from etl_pipeline.config import pipeline_config


In [7]:
from pipelines.ms.ms_pipeline import MSPipeline
from etl_pipeline.config import pipeline_config


In [8]:
with open(f'notebooks/sample/wm_address_in_payload_format.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)
payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [9]:
from etl_pipeline.config import load_agent_configs, pipeline_config

In [10]:
payload = payload_json

In [11]:
engine = JsonProcessingEngine(pipeline_config)
pipeline = MSPipeline(engine, config=pipeline_config)

Field in agent configuration for: COUNTRY_CODE is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY1_CITIZENSHIP is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY_CITIZENSHIP1 is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY1_CITIZENSHIP is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY_CITIZENSHIP1 is not registered field in pipeline.yaml
Field in agent configuration for: COUNTRY_CODE is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY1_CITIZENSHIP is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY_CITIZENSHIP1 is not registered field in pipeline.yaml
Field in agent configuration for: COUNTRY_CODE is not registered field in pipeline.yaml
Field in agent configuration for: TOKENS_RISK_CARRIER is not registered field in pipeline.yaml
Field in agent configur

## transform standardized to cleansed

In [12]:
payload = pipeline.transform_standardized_to_cleansed(payload)

ALL_CONNECTED_ACCOUNT_NAMES# Transform standardized to application

In [13]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

### payload

In [14]:
len(new_payloads)

2

In [15]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload['AP_TRIGGERS']

0    {'Doe': {'ADDRESS1_LINE2': ['Jane Doe AA BBB']...
1    {'Doe': {'ADDRESS1_LINE2': ['Jane Doe AA BBB']...
Name: AP_TRIGGERS, dtype: object

In [16]:
out_payload[[i for i in out_payload.columns if "CARRIER" in i ]].values

array([['Doe'],
       ['Doe']], dtype=object)

In [17]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


PAYLOAD
122438658
0 match
['02/31/1900', '04/31/1910'] ['MAY 6, 1981']
['United States', 'UNITED STATES OF AMERICA'] ['US', 'United States of America']
PAYLOAD
122438659
0 match
['02/31/1900', '04/31/1910'] ['MAY 6, 1981']
['United States', 'UNITED STATES OF AMERICA'] ['Poland', 'PL']


In [18]:
out_payload[[i for i in out_payload.columns if "hit" in i and "agg" in i]]

,ap_all_hit_types_aggregated,wl_all_hit_types_aggregated,all_hit_type_aggregated
0,"[Joe BlackJ, Janusz]","[Joe BlackJ, Janusz]","[{'name': ['PARTY1_NAME_FULL', 'CONCAT_NAME'],..."
1,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."


In [19]:
import pickle
with open("tests/shared/parsed_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [20]:
with open(f'notebooks/sample/big_fat_flat_payload.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)
payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [21]:
from etl_pipeline.config import load_agent_configs, pipeline_config

In [22]:
payload = payload_json

In [23]:
engine = JsonProcessingEngine(pipeline_config)
pipeline = MSPipeline(engine, config=pipeline_config)

Field in agent configuration for: COUNTRY_CODE is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY1_CITIZENSHIP is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY_CITIZENSHIP1 is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY1_CITIZENSHIP is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY_CITIZENSHIP1 is not registered field in pipeline.yaml
Field in agent configuration for: COUNTRY_CODE is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY1_CITIZENSHIP is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY_CITIZENSHIP1 is not registered field in pipeline.yaml
Field in agent configuration for: COUNTRY_CODE is not registered field in pipeline.yaml
Field in agent configuration for: TOKENS_RISK_CARRIER is not registered field in pipeline.yaml
Field in agent configur

## transform standardized to cleansed

In [24]:
payload = pipeline.transform_standardized_to_cleansed(payload)

ALL_CONNECTED_ACCOUNT_NAMES# Transform standardized to application

In [25]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

### payload

In [26]:
len(new_payloads)

9

In [27]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload['WL_MATCHED_TOKENS'], out_payload['TOKENS_RISK_CARRIER']

(0                           ["abouhamza", "abouhamza"]
 1    ["abdalrahmanmilad", "abdalrahmanmilad", "abda...
 2                           ["abdalhadi", "abdalhadi"]
 3                                      ["abdussomada"]
 4                                        ["abouhamza"]
 5                                    ["1177627035211"]
 6                                         ["ahmedoua"]
 7                                ["alfadhli", "udaia"]
 8                                      ["abdussomada"]
 Name: WL_MATCHED_TOKENS, dtype: object,
 0           abouhamza
 1    abdalrahmanmilad
 2           abdalhadi
 3         abdussomada
 4           abouhamza
 5       1177627035211
 6            ahmedoua
 7      alfadhli,udaia
 8         abdussomada
 Name: TOKENS_RISK_CARRIER, dtype: object)

In [28]:
out_payload[[i for i in out_payload.columns if "MATCH" in i ]].values

array([['["abouhamza", "abouhamza"]', 'N'],
       ['["abdalrahmanmilad", "abdalrahmanmilad", "abdalrahmanmilad"]',
        'N'],
       ['["abdalhadi", "abdalhadi"]', 'N'],
       ['["abdussomada"]', 'N'],
       ['["abouhamza"]', 'N'],
       ['["1177627035211"]', 'Y'],
       ['["ahmedoua"]', 'N'],
       ['["alfadhli", "udaia"]', 'N'],
       ['["abdussomada"]', 'N']], dtype=object)

In [29]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


PAYLOAD
94590752
0 match
[] ['01', '01/29/1971', '1971', '29']
['AHMEDOUA'] ['FR', 'FRANCE']
PAYLOAD
94590752
0 match
[] ['27', '07', '07/27/1986', '1986']
['AHMEDOUA'] ['LIBYAN ARAB JAMAHIRIYA', 'LY']
PAYLOAD
94590752
0 match
[] ['06', '07/06/1977', '1977', '07']
['AHMEDOUA'] []
PAYLOAD
94590752
0 match
[] ['17', '08/17/1938', '1938', '08']
['AHMEDOUA'] ['ID', 'INDONESIA']
PAYLOAD
94590752
0 match
[] ['1958', '15', '04/15/1958', '04']
['AHMEDOUA'] ['US', 'UNITED STATES']
PAYLOAD
94590752
0 match
[] []
['AHMEDOUA'] ['RUSSIAN FEDERATION', 'RU']
PAYLOAD
94590752
0 match
[] ['01/01/1982', '01', '1982']
['AHMEDOUA'] ['ML|ML', 'MALI|MALI']
PAYLOAD
94590752
0 match
[] ['27', '08/27/1981', '08', '1981']
['AHMEDOUA'] []
PAYLOAD
94590752
0 match
[] ['17', '08/17/1938', '1938', '08']
['AHMEDOUA'] ['ID', 'INDONESIA']


In [30]:
out_payload[[i for i in out_payload.columns if "hit" in i and "agg" in i]]

,ap_all_hit_types_aggregated,wl_all_hit_types_aggregated,all_hit_type_aggregated
0,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
1,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
2,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
3,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
4,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
5,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
6,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
7,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
8,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."


In [31]:
import pickle
with open("tests/shared/big_fat_payload_parsed.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [32]:
# import pickle
# with open("tests/shared/parsed_payload.pkl", "rb") as f:
#     p = pickle.load(f)

# import pandas as pd
# reference_payload = pd.DataFrame([match for payload in p  for match in payload['watchlistParty']['matchRecords']])



In [33]:
out_payload['TRIGGERED_BY']  = out_payload['TRIGGERED_BY'].apply(sorted)
reference_payload['TRIGGERED_BY']  = reference_payload['TRIGGERED_BY'].apply(sorted)


NameError: name 'reference_payload' is not defined

In [34]:
out_payload['AP_TRIGGERS']  = out_payload['AP_TRIGGERS'].apply(lambda x: {key: x[key] for key in sorted(x)})
reference_payload['AP_TRIGGERS']  = reference_payload['AP_TRIGGERS'].apply(lambda x: {key: x[key] for key in sorted(x)})


NameError: name 'reference_payload' is not defined

In [35]:
for cols in out_payload.columns:
    try:
        pd.testing.assert_series_equal(out_payload[cols], reference_payload[cols])
    except (AssertionError, TypeError):
        reference = remove_nulls_from_aggegated(flatten([i for i in reference_payload[cols].values]))
        output = remove_nulls_from_aggegated(flatten([i for i in out_payload[cols].values]))
        try:
            if isinstance(output, list):
                assert_list(output, reference)
            else:
                assert output == reference
        except AssertionError:
            print((cols, output, reference))
            pass

NameError: name 'reference_payload' is not defined

In [36]:
while True:
    print("remove", columns[-1])
    columns = columns[:-1]
    try:
        pd.testing.assert_frame_equal(out_payload[columns], reference_payload[columns])
        print('ok')
        break
    except AssertionError:
        break
        pass
    except TypeError:
        pass

NameError: name 'columns' is not defined

In [37]:
out_payload[[i for i in out_payload.columns if "hit" in i and "agg" in i]]

,ap_all_hit_types_aggregated,wl_all_hit_types_aggregated,all_hit_type_aggregated
0,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
1,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
2,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
3,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
4,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
5,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
6,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
7,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
8,[],[],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."


In [38]:
with open(f'notebooks/sample/wm_address_in_payload_format_2_input_3_match_records.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [39]:
payload = payload_json

In [40]:
payload = pipeline.transform_standardized_to_cleansed(payload)

In [41]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

In [42]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

,ap_all_dobs_aggregated,wl_all_dobs_aggregated,ap_all_documents_aggregated,wl_all_documents_aggregated,ap_all_company_names_aggregated,wl_all_company_names_aggregated,ap_all_names_aggregated,wl_all_names_aggregated,ap_all_employer_names_aggregated,wl_all_employer_names_aggregated,...,wl_all_hit_types_aggregated,ap_all_matched_tokens_tp_markeds_aggregated,wl_all_matched_tokens_tp_markeds_aggregated,ap_all_ap_name_tp_markeds_aggregated,wl_all_ap_name_tp_markeds_aggregated,ap_all_ap_id_tp_markeds_aggregated,wl_all_ap_id_tp_markeds_aggregated,ap_all_is_entity_matchs_aggregated,wl_all_is_entity_matchs_aggregated,all_hit_type_aggregated
0,"[02/31/1900, 04/31/1910]","[MAY 6, 1981]","[154421273, 022368917]",[],"[Eva Pladimirova, Pladimir Vutin]",[Joe Ding],"[Eva Pladimirova, Pladimir Vutin]",[Joe Ding],[],[Joe Ding],...,[],[],[],[],[],[],[],[I],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
1,"[02/31/1900, 04/31/1910]","[MAY 6, 1981]","[154421273, 022368917]",[],"[Eva Pladimirova, Pladimir Vutin]",[Joe Ding],"[Eva Pladimirova, Pladimir Vutin]",[Joe Ding],[],[Joe Ding],...,[],[],[],[],[],[],[],[I],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
2,"[02/31/1900, 04/31/1910]","[MAY 6, 1981]","[154421273, 022368917]",[],"[Eva Pladimirova, Pladimir Vutin]",[Joe Dong],"[Eva Pladimirova, Pladimir Vutin]",[Joe Dong],[],[Joe Dong],...,[],[],[],[],[],[],[],[I],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."


In [43]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


PAYLOAD
123
0 match
['02/31/1900', '04/31/1910'] ['MAY 6, 1981']
['United States', 'UNITED STATES OF AMERICA'] ['US', 'United States of America']
PAYLOAD
122438659
0 match
['02/31/1900', '04/31/1910'] ['MAY 6, 1981']
['United States', 'UNITED STATES OF AMERICA'] ['US', 'United States of America']
PAYLOAD
122438659
0 match
['02/31/1900', '04/31/1910'] ['MAY 6, 1981']
['United States', 'UNITED STATES OF AMERICA'] ['Poland', 'PL']


In [44]:
import pickle
with open("tests/shared/parsed_payload_2_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [45]:
new_dict = {}
def return_key(dict_, prefix):
    
    if isinstance(dict_, list):
        for num, i in enumerate(dict_):
            return_key(i, prefix +f"[{num}]")
        return
    if isinstance(dict_, str) or dict_ is None:
        new_dict[prefix] = dict_
        return

    for key in dict_:
        if prefix:
            basic_prefix = prefix + "." + key
        else:
            basic_prefix = key
        return_key(dict_[key], basic_prefix)

In [46]:
with open(f'notebooks/sample/wm_party_payload_without_supplemental_info.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [47]:
payload = payload_json

In [48]:
payload = pipeline.transform_standardized_to_cleansed(payload)

In [49]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

In [50]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

,ap_all_dobs_aggregated,wl_all_dobs_aggregated,ap_all_documents_aggregated,wl_all_documents_aggregated,ap_all_company_names_aggregated,wl_all_company_names_aggregated,ap_all_names_aggregated,wl_all_names_aggregated,ap_all_employer_names_aggregated,wl_all_employer_names_aggregated,...,wl_all_hit_types_aggregated,ap_all_matched_tokens_tp_markeds_aggregated,wl_all_matched_tokens_tp_markeds_aggregated,ap_all_ap_name_tp_markeds_aggregated,wl_all_ap_name_tp_markeds_aggregated,ap_all_ap_id_tp_markeds_aggregated,wl_all_ap_id_tp_markeds_aggregated,ap_all_is_entity_matchs_aggregated,wl_all_is_entity_matchs_aggregated,all_hit_type_aggregated
0,[],"[MAY 6, 1981]",[],[],[],[Joe Ding],[],[Joe Ding],[],[Joe Ding],...,[],[],[],[],[],[],[],[C],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
1,[],"[MAY 6, 1981]",[],[],[],[Joe Dong],[],[Joe Dong],[],[Joe Dong],...,[],[],[],[],[],[],[],[C],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."


In [51]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


PAYLOAD
122438658
0 match
[] ['MAY 6, 1981']
[] []
PAYLOAD
122438659
0 match
[] ['MAY 6, 1981']
[] ['Poland', 'PL']


In [52]:
import pickle
with open("tests/shared/empty_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [53]:
from etl_pipeline.config import load_agent_config

In [54]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

,ap_all_dobs_aggregated,wl_all_dobs_aggregated,ap_all_documents_aggregated,wl_all_documents_aggregated,ap_all_company_names_aggregated,wl_all_company_names_aggregated,ap_all_names_aggregated,wl_all_names_aggregated,ap_all_employer_names_aggregated,wl_all_employer_names_aggregated,...,wl_all_hit_types_aggregated,ap_all_matched_tokens_tp_markeds_aggregated,wl_all_matched_tokens_tp_markeds_aggregated,ap_all_ap_name_tp_markeds_aggregated,wl_all_ap_name_tp_markeds_aggregated,ap_all_ap_id_tp_markeds_aggregated,wl_all_ap_id_tp_markeds_aggregated,ap_all_is_entity_matchs_aggregated,wl_all_is_entity_matchs_aggregated,all_hit_type_aggregated
0,[],"[MAY 6, 1981]",[],[],[],[Joe Ding],[],[Joe Ding],[],[Joe Ding],...,[],[],[],[],[],[],[],[C],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
1,[],"[MAY 6, 1981]",[],[],[],[Joe Dong],[],[Joe Dong],[],[Joe Dong],...,[],[],[],[],[],[],[],[C],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."


In [55]:
from omegaconf import OmegaConf
alert_agents_config = {
        "alert_type": OmegaConf.load(os.path.join(CONFIG_APP_DIR, "agents", "agents.yaml"))
    }


import omegaconf
def get_fields(dict_):
    fields = []
    for key in dict_:
        
        if isinstance(dict_[key], omegaconf.dictconfig.DictConfig):
            new_fields = get_fields(dict_[key])
            fields.extend(new_fields)
        else:
#             import pdb; pdb.set_trace()
            fields.extend(dict_[key])
    return fields

fields = [i.split('.')[-1] for i in get_fields(alert_agents_config['alert_type']) if "alertedParty.inputRecordHist.inputRecords.INPUT_FIELD" not in i]
    
    

from glob import glob

files = glob("pipelines/**/*.py",  recursive=True)
files.extend(glob("etl_pipeline/**/*.py",  recursive=True))

fields_to_check = list(fields)
for file in files:
    with open(file) as f:
        text = f.read()
    new_fields_to_check = []
    
    for field in fields_to_check:
        if field in text:
            continue
        new_fields_to_check.append(field)
    fields_to_check = new_fields_to_check
        
    

fields_to_check

CONFIG_APP_DIR = os.environ["CONFIG_APP_DIR"] 

NameError: name 'CONFIG_APP_DIR' is not defined

In [56]:
import pickle

In [57]:
with open("/tmp/request.pkl", 'rb') as f:
    r = pickle.load(f)

In [58]:
from silenteight.datasource.api.name.v1.name_pb2 import (
    NameFeatureInput,
)

In [59]:
from google.protobuf.any_pb2 import Any

In [60]:
some_any2 = Any()
message = NameFeatureInput()
some_any2.Pack(NameFeatureInput())
# some_any2.CopyFrom(i.feature_inputs[0].agent_feature_input)


In [61]:
i.feature_inputs[0].agent_feature_input

NameError: name 'i' is not defined

In [62]:
i.feature_inputs[0].agent_feature_input.Unpack(message)

NameError: name 'i' is not defined

In [63]:
message

In [64]:
unpacked_msg = extension_pb2.GeneralContent_Extension())


SyntaxError: invalid syntax (<ipython-input-64-45f018d90cca>, line 1)

In [65]:
NameFeatureInput.ParseFromString()

TypeError: descriptor 'ParseFromString' of 'google.protobuf.pyext._message.CMessage' object needs an argument

In [66]:
from sile
i.feature_inputs[0].agent_feature_input.value.decode('utf-8')

SyntaxError: invalid syntax (<ipython-input-66-5fc39aa70bf1>, line 1)

In [67]:
type(i.feature_inputs[0].agent_feature_input.type_url)

NameError: name 'i' is not defined

In [ ]:
for i in response.agent_inputs:
    print(i.feature_inputs)

In [ ]:
for 